In [1]:
#get document frequency for terms in the same article/instance; and get each sentence's BOW; and get each question's BOW;
#get each sentence's length; get each article's average length of sentences
import csv
import json
import copy
import os
import nltk
from math import log
from collections import defaultdict, Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.spatial.distance import cosine as cos_distance
from sklearn.decomposition import TruncatedSVD
from nltk.tag import StanfordNERTagger
def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word] = BOW.get(word,0) + 1
    return BOW
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma
def input_data():
    base_path = os.path.join('data/')
    train_file = base_path + 'QA_train.json'
    train_data = json.load(open(train_file))
    test_file = base_path + 'QA_test.json'
    test_data = json.load(open(test_file))
    dev_file = base_path + 'QA_dev.json'
    dev_data = json.load(open(dev_file))

    return train_data,test_data,dev_data
def transform_text(text):
    text = nltk.word_tokenize(text)
    text = [lemmatize(word.lower()) for word in text]
    result = []
    for word in text:
        if word not in stopwords and word not in punctuations:
            result.append(word)
    return result
def get_Docfrequency_SentenceBOW(dataset):
    #save dics, each dictionary contains document frequencies for all terms in the same article
    question_list = []
    #save lists, each list represent an article, saving sentences' bow
    total_sentence_bow = []
    #save lists, each list represent an article, saving questions' bow
    total_question_bow = []
    #save lists, each list represent all sentences' lengthes.
    sent_lengthes = []
    #save a list, each item represents the average length of sentences
    avg_lengthes = []
    #
    answer_id = []
    
    for article in dataset:
        #Docfrequency
        article_dic = defaultdict(list)
        keyterms = [] #save all distinct terms in questions
        
        #SentenceBOW
        bow_list = []
        
        #QuestionBOW
        que_list = []
        
        #SentenceLength
        sent_len = []
        
        #TotalLength
        total_len = 0
        
        #RightAnser
        right_answer = []
        
        qas = article['qa']
        sentences = article['sentences']
        for qa in qas:
            question = qa['question']
            newquestion = transform_text(question)
            #QuestionBOW
            que_list.append(get_BOW(newquestion))
            answer = qa['answer_sentence']
            right_answer.append(answer)
            
            keyterms.extend(newquestion)
        keyterms = set(keyterms)
        
        #save sentences' BOW in list sen_BOW
        sen_words = []
        for sent in sentences:
            sent = transform_text(sent)
            #Docfrequency
            sen_words.append(sent)
            
            #SentenceBOW
            bow_list.append(get_BOW(sent))
            
            #SentenceLength
            sent_len.append(len(sent))
            
            #TotalLength
            total_len += len(sent)
            
        
        #calculate doc frequency    
        for term in keyterms:
            for i,bow in enumerate(sen_words):
                if term in bow:
                    article_dic[term].append(i)
                    
        #Docfrequency
        question_list.append(article_dic)
        #SentenceBOW
        total_sentence_bow.append(bow_list)
        #QuestionBOW
        total_question_bow.append(que_list)
        #SentenceLength
        sent_lengthes.append(sent_len)
        #AverageLength
        avg_lengthes.append(float(total_len)/len(sentences))
        #
        answer_id.append(right_answer)
        
    return question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes, answer_id

train,test,dev = input_data()
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
punctuations = [',','\'\'','?','\'','.','%','(',')',';','``']
question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes, answer_id_list= get_Docfrequency_SentenceBOW(train)

In [2]:
import math
def BM25(articles_index,k1,k2,b):
    total_queries = len(total_question_bow[articles_index])
    count = 0
    correct_id = []
    for index in range(len(total_question_bow[articles_index])):
        answer_id = answer_id_list[articles_index][index]
        guess_id = find_max_score_sentence(articles_index,index,k1,k2,b)
        if answer_id == guess_id:
            correct_id.append(index)
            count += 1   
    accurancy = float(count)/total_queries  
    return correct_id,accurancy

def find_max_score_sentence(articles_index,index,k1,k2,b):

    query_dict = total_question_bow[articles_index][index]
    max_score = 0
    guess_sentence = 0
    for index in range(len(total_sentence_bow[articles_index])):     
        score = 0  
        sentence_dict = total_sentence_bow[articles_index][index]
        for word in query_dict:
            document_fre_list = question_list[articles_index].get(word,None)
            
            
            N = len(total_sentence_bow[articles_index])
            n_qi = 0
            if document_fre_list != None:
                n_qi = len(document_fre_list)
            else:
                n_qi = 0
            fi = sentence_dict.get(word,0)
            qfi = query_dict.get(word,0)
            dl = sent_lengthes[articles_index][index]
            avgdl = avg_lengthes[articles_index]
            
            K = k1*(1-b+b*(float(dl)/avgdl)) 
            W = math.log((N-n_qi+0.5)/(n_qi+0.5))
            R = (fi*(k1+1))/(fi+K)*qfi*(k2+1)/(qfi+k2)
            score += W*R
        if score > max_score:
            max_score = score
            guess_sentence = index
    return guess_sentence

In [3]:
accuracy = 0.0

k1_list = [1.1]
k2_list = [0]
b_list = [0.18]

correct = []

test_length = len(train)
for k1 in k1_list:
    for k2 in k2_list:
        for b in b_list:
            accuracy = 0.0
            for i in range(0,test_length):
                correct_id,i_accuracy = BM25(i,k1,k2,b)
                accuracy += i_accuracy
                correct.append(correct_id)
            average_accuracy = accuracy/test_length
            print "Accuracy of BM25 with k1:",k1," k2:",k2," b:",b
            print average_accuracy

Accuracy of BM25 with k1: 1.1  k2: 0  b: 0.18
0.683639289744


In [ ]:
# Accuracy of BM25 with k1: 1.1  k2: 0  b: 0.18
# 0.683639289744

In [10]:
import re
pattern = re.compile(r'[0-9]+s') 
match = pattern.search('mid-1950s') 

text = u'Ccording to the Federal Bureau of Investigation\'s Uniform Crime Reports, in 2010 the Raleigh Police Department and other agencies in the city reported 1,740 incidents of violent crime and 12,995 incidents of property crime – far below both the national average and the North Carolina average.'
x = re.split(r'("|\?|;|\\u2013|, |\(|\s|\))\s*', text)

print text[0].isupper()

for word in x:
    if word == u'\u2013':
        print 'ssss'
print x


True
ssss
[u'Ccording', u' ', u'to', u' ', u'the', u' ', u'Federal', u' ', u'Bureau', u' ', u'of', u' ', u"Investigation's", u' ', u'Uniform', u' ', u'Crime', u' ', u'Reports', u', ', u'in', u' ', u'2010', u' ', u'the', u' ', u'Raleigh', u' ', u'Police', u' ', u'Department', u' ', u'and', u' ', u'other', u' ', u'agencies', u' ', u'in', u' ', u'the', u' ', u'city', u' ', u'reported', u' ', u'1,740', u' ', u'incidents', u' ', u'of', u' ', u'violent', u' ', u'crime', u' ', u'and', u' ', u'12,995', u' ', u'incidents', u' ', u'of', u' ', u'property', u' ', u'crime', u' ', u'\u2013', u' ', u'far', u' ', u'below', u' ', u'both', u' ', u'the', u' ', u'national', u' ', u'average', u' ', u'and', u' ', u'the', u' ', u'North', u' ', u'Carolina', u' ', u'average.']


In [22]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import re
import sys

location_list = ['country','county']
number_list = ['msa','far','much','year','era','ration','years','time','many','population','district','large','percent','average','day','decade','big','long']
time_list = ['january','february','march','april','may','june','july','august','september','october','november','december','hours','minutes','miles','inches','foot','years','seasons']
name_list = ['university','name','center','president','denominations','denomination','film','broadcaster','pitcher','commentator']

def token(text):
    text = text[:-1]
    result = []
    combo = u''
    flag_upper = False
    x = re.split(r'("|\?|;|, |\(|\s|\))\s*', text)
    #print x
    for i,word in enumerate(x):
        if word != u'' and word != u' ':
            if word == u', ':
                word = u','
            if word == u'\u2013':
                word = u'-'
            if word.lower() in ['in','on','after','at']:
                word = word.lower()
            if word[0].isalpha():
                if flag_upper == False and word[0].isupper():
                    combo = word
                    flag_upper = True
                elif flag_upper == False and word[0].islower():
                    result.append(word)
                elif flag_upper == True and word[0].isupper():
                    combo = combo + u' ' + word
                elif flag_upper == True and word[0].islower():
                    result.append(combo)
                    combo = u''
                    result.append(word)
                    flag_upper = False
            else:
                if flag_upper == True:
                    result.append(combo)
                    combo = u''
                    result.append(word)
                    flag_upper = False
                else:
                    result.append(word)
    if combo != u'':
        result.append(combo) 
    return result
def token_question(text):
    text = text[:-1]
    result = []
    combo = u''
    flag_upper = False
    x = re.split(r'("|\?|;|, |\(|\s|\))\s*', text)
    for i,word in enumerate(x):
        if i == 0:
            result.append(word)
        else:
            if word == u', ':
                word = u','
            if word != u'' and word != u' ':
                if word[0].isalpha():
                    if flag_upper == False and word[0].isupper():
                        combo = word
                        flag_upper = True
                    elif flag_upper == False and word[0].islower():
                        result.append(word)
                    elif flag_upper == True and word[0].isupper():
                        combo = combo + u' ' + word
                    elif flag_upper == True and word[0].islower():
                        result.append(combo)
                        combo = u''
                        result.append(word)
                        flag_upper = False
                else:
                    if flag_upper == True:
                        result.append(combo)
                        combo = u''
                        result.append(word)
                        flag_upper = False
                    else:
                        result.append(word)
    if combo != u'':
        result.append(combo) 
    return result
def transfer_pos_question(pos):
    new_pos = []
    for (word,wtype) in pos:
        if word.lower() == 'what' or word.lower() == 'what\'s':
            new_pos.append((word,'WHAT'))
        elif word.lower() == 'do' or word.lower() == 'does' or word.lower() == 'did':
            new_pos.append((word,'DO'))
        elif word.lower() in number_list:
            new_pos.append((word,'TIME'))
        elif word.lower() == 'is' or word.lower() == 'was' or word.lower() == 'are' or word.lower() == 'were' or word.lower() == 'be':
            new_pos.append((word,'BE'))
        elif word.lower() in location_list:
            new_pos.append((word,'LOC'))
        elif word.lower() == 'when':
            new_pos.append((word,'WHEN'))
        elif word.lower() == 'where':
            new_pos.append((word,'WHERE'))
        elif word.lower() == 'can':
            new_pos.append((word,'CAN'))
        elif word.lower() == 'how':
            new_pos.append((word,'HOW'))
        elif word.lower() == 'who' or word.lower() == 'whom' or word.lower() == 'whose'  or word.lower() == 'whos':
            new_pos.append((word,'WHO'))
        elif word.lower() == 'which':
            new_pos.append((word,'WHICH'))
        elif word.lower() in name_list:
            new_pos.append((word,'NAME'))
        elif word.lower() == 'define':
            new_pos.append((word,'DEFINE'))
        elif word.lower() == 'should':
            new_pos.append((word,'SHOULD'))
        elif word.lower() == 'why' or word.lower() == 'wy':
            new_pos.append((word,'WHY'))
        else:
            new_pos.append((word,wtype))
    return new_pos
def transfer_pos_sentence(pos):
    new_pos = []
    
    for (word,wtype) in pos:
        if word.lower() == 'and' or word.lower() == 'or':
            new_pos.append((word,'POSICC'))
        elif word.lower() == 'but' or word.lower() == 'without':
            new_pos.append((word,'NEGCC'))
        elif word.lower() == 'with':
            new_pos.append((word,'WITH'))
        elif word.lower() == 'a' or word.lower() == 'an':
            new_pos.append((word,'A'))
        elif word == '"':
            new_pos.append((word,'"'))
        elif word.lower() in time_list or '$' in word:
            new_pos.append((word,'TIME'))
        elif word == 'around':
            new_pos.append((word,'AROUND'))
        elif word == 'mm':
            new_pos.append((word,'CD'))   
        elif word == 'by':
            new_pos.append((word,'BY'))
        elif word[0].isupper():
            new_pos.append((word,'UPPER'))
        else:
            new_pos.append((word,wtype))
    return new_pos
def get_continuous_chunks(text,texttype):
    #text = text.encode('ascii','replace')
    t = copy.deepcopy(text)
    #print token(t)
    if texttype==0:
        pos = pos_tag(token(t))
        pos = transfer_pos_sentence(pos)
        grammar = r"""
                    BASICN:
                        {<UPPER>}
                        {<NN.*>}
                    J:
                        {<JJ.*><VBN>}
                        {<JJ.*><POSICC><JJ.*>}   
                        {<JJ.*>+}
                        {<NN.*><POS>}
                    N:
                        {<DT>?<BASICN>?<J>?<BASICN>+}
                        <\">{<A>?<J>?<BASICN>+}<\">
                    COMBON:
                        {(<N><,>)*<N><,>?<POSICC><N>}
                    NWC:
                        {<N><WITH><COMBON>}
                    """
    elif texttype==1:
        pos = pos_tag(token_question(t))
        pos = transfer_pos_question(pos)
        #print pos
        grammar = r"""
                    WHAT: 
                        {<WHAT>}
                        {<WHICH>}
                        {<DEFINE>}
                    WHO:
                        {<WHO>}
                        {<WHAT><BE>?<DT>?<JJ|RB>*<NAME>}
                        {<WHAT><JJ|RB>*<NN>+<NAME>}
                    WHEN:
                        {<WHICH><TIME>}
                        {<HOW><TIME>}
                        {<WHAT><BE>?<DT>?<JJ>?<NN>*<JJ>?<TIME>}
                        {<WHEN>}
                    WHERE:
                        {<WHERE>}
                        {<WHAT><LOC>}
                    HOW:
                        {<CAN>}
                        {<DO>}
                        {<SHOULD>}
                        {<WHY>}
                        {<HOW>}
                    
                    """
    elif texttype==2:
        #when
        pos = pos_tag(token(t))
        pos = transfer_pos_sentence(pos)
        #print pos
        grammar = r"""
                    C:
                        {<TIME><CD>?<,><CD>}
                        {<TIME><CD>?<,><TIME>}
                        {<CD><TIME>}
                        {<CD><\.><CD>}
                        {<AROUND><CD>+}
                        {<RB><TIME>+}   
                        {<JJ><TIME>+}
                        {<AROUND><TIME>+}
                        {<TIME>+<CD>+}
                        {<DT>?<TIME>+}
                        {<CD>+}      
                    COMBOC:
                        {<C><POSICC><C>}
                    """
    elif texttype==3:
        #when
        pos = pos_tag(token(t))
        pos = transfer_pos_sentence(pos)
        #print pos
        grammar = r"""
                    H:
                        {<RB><JJ><IN>?}
                        {<IN>?<JJ.*>*<NN.*>+}
                    HH:
                        {<H>+}
                    SEN:
                        {<.*>+}
                        }<VBD|BY|,|WITH|NEGCC|TO>+{
                    """
    elif texttype==4:
        #who
        pos = pos_tag(token(t))
        pos = transfer_pos_sentence(pos)
        #print pos
        grammar = r"""
                    UP:
                        {<DT>?<UPPER>}
                    SEN:
                        {<.*>+}
                        }<VBD|BY|,|WITH>+{
                    """
    cp = nltk.RegexpParser(grammar) 
    result = []
    poss = copy.deepcopy(pos)
    tree = cp.parse(pos)
    #record the position of pos
    flag = 0
    for subtree in tree.subtrees():
        if subtree.label() != 'S':
            phrase = u''
            for word,pos in subtree.leaves():
                if word == ',':
                    phrase = phrase + word
                else:
                    phrase = phrase + u' '
                    phrase = phrase + word
            result.append((subtree.label(),phrase[1:]))         
    return poss,result

In [5]:
from nltk.tag import StanfordNERTagger
def input_NER():
    stanford_dir = os.path.join('stanford-ner-2016-10-31')
    jarfile = os.path.join(stanford_dir,'stanford-ner.jar')
    modelfile = os.path.join(stanford_dir,'classifiers/english.muc.7class.distsim.crf.ser.gz')
    return modelfile,jarfile
model,jar = input_NER()
st = StanfordNERTagger(model,jar)

In [6]:
def rank_rule_1(entity,query):
    #lower scores for content words also appear in the query
    count = 0
    length = len(entity)
    for word in entity:
        word = lemmatize(word.lower())
        if word not in stopwords:
            if word in query:
                count += 1
    score = 1 - float(count)/length
    return score

def rank_rule_2(ner_entity,query):
    #answers which match the question type should be ranked higher
    question_word = get_question_word(query)
    question_type = get_question_type(question_word,query)
    if ner_entity == question_type:
        score = 1
    else:
        score = 0
    return score

In [7]:
def get_NER_entity(ner_sentence,type):
    result = []
    temp = ''
    for index in range(len(ner_sentence)):
        if ner_sentence[index][1] == type:
            if temp != '':
                temp += ' '
            temp += ner_sentence[index][0]
            if index + 1 < len(ner_sentence):
                if ner_sentence[index+1][1] != type:
                    result.append(temp)
                    temp = ''
            else:
                result.append(temp)
    return result

In [8]:
def get_answer_list(flag,wresult,answer_list):
    #if flag == True:
    for t,w in wresult:
        if t != u'J':
            answer_list.append(w)
    return answer_list

In [26]:
total_count = 0.0
total_question = 0.0
#for index,article in enumerate(train[:1]):
for index in range(10):
    article = train[index]
    qas = article['qa']
    sentences = article['sentences']
    token_sentences = copy.deepcopy(sentences)
    for i in range(len(sentences)):
        token_sentences[i] = nltk.word_tokenize(token_sentences[i])
    ner_sentences = st.tag_sents(token_sentences)
    count = 0
    for i in correct[index]:
        answer_list = []
        qa = qas[i]
        #print '---------------'
        flag = False
        text_question = qa['question']
        answer = qa['answer']
        answer_sentence = sentences[qa['answer_sentence']]
        pos,result = get_continuous_chunks(text_question,1)
        #print pos
        if result != []:
            wtype,word = result[0]
            total_question += 1
            if wtype == 'WHAT':          
                wpos,wresult = get_continuous_chunks(answer_sentence,0)
                for wwtype,wword in wresult:
                    if qa['answer'] == wword:
                        flag = True
                        total_count += 1
                answer_list = get_answer_list(flag,wresult,[])

            elif wtype == 'WHEN':
                wpos,wresult = get_continuous_chunks(answer_sentence,2)
                for wwtype,wword in wresult:
                    if qa['answer'] == wword:
                        flag = True
                        total_count += 1
                answer_list = get_answer_list(flag,wresult,[])

            elif wtype == 'HOW':               
                wpos,wresult = get_continuous_chunks(answer_sentence,3)           
                for wwtype,wword in wresult:
                    if qa['answer'] == wword:
                        flag = True
                        total_count += 1
                answer_list = get_answer_list(flag,wresult,[])
            elif wtype == 'WHO':
                ner_answer_sentence = ner_sentences[qa['answer_sentence']]
                answer_list = get_NER_entity(ner_answer_sentence,'PERSON')
                if answer_list == []:
                    #or do some other things
                    answer_list = get_NER_entity(ner_answer_sentence,'ORGANIZATION')
                if answer_list == []:
                    wpos,wresult = get_continuous_chunks(answer_sentence,4)
                    answer_list = get_answer_list(flag,wresult,[])

                if qa['answer'] in answer_list:
                    flag = True
                    total_count += 1

            elif wtype == 'WHERE':
                ner_answer_sentence = ner_sentences[qa['answer_sentence']]
                answer_list = get_NER_entity(ner_answer_sentence,'LOCATION')
                if answer_list == []:
                    wpos,wresult = get_continuous_chunks(answer_sentence,4)
                    answer_list = get_answer_list(flag,wresult,[])
                if qa['answer'] in answer_list:
                    flag = True
                    total_count += 1
    #             print '^'*100
    #             print 'list: ',answer_list
    #             print 'A: ',qa['answer']
            '''if flag == False:
                print '^'*100
                print text_question
                #print pos,result
                print wtype
                print answer_sentence
                print transfer_pos_sentence(pos_tag(token(answer_sentence)))
                print answer
                print answer_list'''
        else:
            wpos,wresult = get_continuous_chunks(answer_sentence,0)
            for wwtype,wword in wresult:
                if qa['answer'] == wword:
                    flag = True
                    total_count += 1
            answer_list = get_answer_list(flag,wresult,[])
            #print '^'*100
            #print text_question
            #print answer
            #print pos
        '''if answer_list != []:
            query = copy.deepcopy(text_question)
            query = nltk.word_tokenize(query)
            for index in range(len(query)):
                query[index] = lemmatize(query[index].lower())
            scores_1 = []
            for entity in answer_list:
                entity = nltk.word_tokenize(entity) 
                #score = rank_rule_1(entity,query)
                #scores_1.append(score)
            print '^'*100
            print 'Q: ',text_question
            print 'A: ',qa['answer']
            print answer_list
            print answer_sentence
            print 'socres: ',scores_1'''
print total_count/total_question   

0.728184553661
